In [1]:
import rebound
import reboundx
import numpy as np

In [2]:
sim = rebound.Simulation()
sim.add(m=1.,r=0.2)
sim.add(a=8.,e=0.9999, f=np.pi)
sim.add(a=16.)
ps = sim.particles
rebx = reboundx.Extras(sim)
gr = rebx.add("gr")
gr.params["c"] = 3.e8

In [3]:
import numpy as np
test = np.array([[[1,2], [3,4], [5,6]], [[7,8], [9,10], [11,12]], [[13,14], [15,16], [17,18]], [[19,20], [21,22], [23,24]]])
ps[1].params["test"] = test

In [4]:
print(ps[1].params["test"])

[[[ 1  2]
  [ 3  4]
  [ 5  6]]

 [[ 7  8]
  [ 9 10]
  [11 12]]

 [[13 14]
  [15 16]
  [17 18]]

 [[19 20]
  [21 22]
  [23 24]]]


In [5]:
rebx.save("test.bin")

In [6]:
sim = rebound.Simulation()
sim.add(m=1.,r=0.2)
sim.add(a=8.,e=0.9999, f=np.pi)
sim.add(a=16.)
ps = sim.particles

In [7]:
rebx2 = reboundx.Extras.from_file(sim, "test.bin")

In [8]:
gr = rebx2.get_effect("gr")

In [9]:
gr.params["c"]

300000000.0

In [10]:
ps[1].params["test"]

array([[[ 1,  2],
        [ 3,  4],
        [ 5,  6]],

       [[ 7,  8],
        [ 9, 10],
        [11, 12]],

       [[13, 14],
        [15, 16],
        [17, 18]],

       [[19, 20],
        [21, 22],
        [23, 24]]], dtype=int32)

In [24]:
test.shape

(4, 3, 2)

In [4]:
import numpy as np
o1 = sim.particles[1].orbit
o2 = sim.particles[2].orbit
objndarray = np.array([[[o1,o2],[o2,o1],[o1,o1]], [[o1,o2],[o2,o1],[o1,o1]],[[o1,o2],[o2,o1],[o1,o1]],[[o1,o2],[o2,o1],[o1,o1]]], dtype=object)

In [17]:
print(objndarray[0][1][0])

<rebound.Orbit instance, a=15.999999999999996 e=1.3704315460216779e-16 inc=0.0 Omega=0.0 omega=3.133296326794899 f=-1.570796326794898>


In [5]:
sim.particles[1].params["objndarray"] = objndarray

In [6]:
q = sim.particles[1].params["objndarray"] 

<rebound.Orbit instance, a=8.000000000001613 e=0.9999000000000001 inc=0.0 Omega=0.0 omega=0.0 f=3.133732126499554>
<rebound.Orbit instance, a=15.999999999999996 e=1.3704315460216779e-16 inc=0.0 Omega=0.0 omega=3.133296326794899 f=-1.570796326794898>
<rebound.Orbit instance, a=15.999999999999996 e=1.3704315460216779e-16 inc=0.0 Omega=0.0 omega=3.133296326794899 f=-1.570796326794898>
<rebound.Orbit instance, a=8.000000000001613 e=0.9999000000000001 inc=0.0 Omega=0.0 omega=0.0 f=3.133732126499554>
<rebound.Orbit instance, a=8.000000000001613 e=0.9999000000000001 inc=0.0 Omega=0.0 omega=0.0 f=3.133732126499554>
<rebound.Orbit instance, a=8.000000000001613 e=0.9999000000000001 inc=0.0 Omega=0.0 omega=0.0 f=3.133732126499554>
<rebound.Orbit instance, a=8.000000000001613 e=0.9999000000000001 inc=0.0 Omega=0.0 omega=0.0 f=3.133732126499554>
<rebound.Orbit instance, a=15.999999999999996 e=1.3704315460216779e-16 inc=0.0 Omega=0.0 omega=3.133296326794899 f=-1.570796326794898>
<rebound.Orbit insta

In [11]:
print(q[0][1][0])

<rebound.Orbit instance, a=4.0 e=0.7000000000000001 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [13]:
q.shape == objndarray.shape

True

In [15]:
for i in q.flatten():
    print(i)

<rebound.Orbit instance, a=1.0000000000000002 e=0.20000000000000018 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=4.0 e=0.7000000000000001 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=4.0 e=0.7000000000000001 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=1.0000000000000002 e=0.20000000000000018 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=1.0000000000000002 e=0.20000000000000018 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=1.0000000000000002 e=0.20000000000000018 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=1.0000000000000002 e=0.20000000000000018 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=4.0 e=0.7000000000000001 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=4.0 e=0.7000000000000001 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=1.0000000000000002 e=0.20000000000000018 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=1.00000

In [4]:
sim.particles[1].params["a"] = [1.,2.,3.,4.]

In [5]:
sim.particles[1].params["a"]

array([ 1.,  2.,  3.,  4.])

In [6]:
q = sim.particles[1].params["a"]

In [7]:
q[2] = 927

In [8]:
sim.particles[1].params["a"]

array([   1.,    2.,  927.,    4.])

In [9]:
sim.particles[1].params["test"] = [ps[1].orbit, ps[2].orbit]

In [12]:
q = sim.particles[1].params["test"]

In [13]:
print(q[1])

<rebound.Orbit instance, a=4.0 e=0.7000000000000001 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [14]:
q[1].a = 7

In [15]:
print(sim.particles[1].params["test"][1])

<rebound.Orbit instance, a=7.0 e=0.7000000000000001 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [16]:
q.contents._length_

4

In [18]:
import numpy as np
a = np.array([q.contents[i] for i in range(q.contents._length_)], dtype=object)

In [19]:
print(a)

[1.0 7.0 3.0 4.0]


In [14]:
q.contents

In [6]:
print(sim.particles[1].params["test"])

<rebound.Orbit instance, a=4.0 e=0.7000000000000001 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [18]:
test = [sim.particles[1].orbit, sim.particles[2].orbit]

In [19]:
sim.particles[1].params["array"] = test

In [20]:
q= sim.particles[1].params["array"]

In [25]:
q = np.array([1.,2.,3.,4.], dtype=object)

In [26]:
q.dtype

dtype('O')

In [9]:
import numpy as np
a = np.array(test)

In [21]:
type(a[0]).__name__

'void'

In [26]:
type(a[0])

numpy.void

In [10]:
tempval = a

In [11]:
try:
    while True:
        tempval = tempval[0]
        if type(tempval).__name__ == 'void':
            print('hi')
except:
    print(tempval)

hi
0.8


In [30]:
try:
    while True:
        tempval = tempval[0]
        if type(tempval).__name__ == 'void':
            print('hi')
except:
    print(tempval)

SyntaxError: invalid syntax (<ipython-input-30-605ae39af4f2>, line 2)

In [8]:
import numpy as np
ps[1].params["list"] = test
ps[1].params["array"] = np.array(test, dtype=object)

In [15]:
ps[1].params["list"][0].e == ps[1].params["array"][0].e

True

In [19]:
getattr(ps[1].params["list"][0], 'e')

0.20000000000000018

In [26]:
for a in [a for a in dir(rebound.Orbit) if not a.startswith('_')]:
    print(getattr(ps[1].params["list"][0], a) == getattr(ps[1].params["array"][0], a))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [33]:
type(ps[1].orbit) == rebound.Orbit

True

In [1]:
import numpy as np
b = np.array([[[1.,2.,3.],[4.,5.,6.]], [[1.,2.,3.],[4.,5.,6.]],[[1.,2.,3.],[4.,5.,6.]], [[1.,2.,3.],[4.,5.,    6.]]])

In [2]:
b

array([[[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]],

       [[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]],

       [[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]],

       [[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]]])

In [5]:
ps[1].params["bah"] = b

In [8]:
q[1,1,1] = 29382

In [7]:
q = ps[1].params["bah"]

In [9]:
ps[1].params["bah"]

array([[[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]],

       [[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]],

       [[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]],

       [[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]]])

In [31]:
type(ps[1].params["list"][0])

rebound.simulation.Orbit

In [27]:
print(ps[1].params["list"][0] == ps[1].orbit)

False


In [30]:
True == 1

True

In [28]:
print(ps[1].params["array"][0])

<rebound.Orbit instance, a=1.0000000000000002 e=0.20000000000000018 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [29]:
print(ps[1].orbit)

<rebound.Orbit instance, a=1.0000000000000002 e=0.20000000000000018 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [16]:
print(ps[0].params["orbits"][1])

<rebound.Orbit instance, a=4.0 e=0.7000000000000001 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [18]:
b = a(*c)

In [13]:
test[0].e = 37

In [26]:
import numpy as np
a = np.array([[1.,2.],[3.,4.],[5.,6.]])

In [29]:
[i for i in a.flatten()]

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

In [30]:
a.size

6

In [10]:
print(q[0])

<rebound.Orbit instance, a=1.0000000000000002 e=0.20000000000000018 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [ ]:
q = sim.particles[1].params["array"]

In [15]:
type(test).__name__

'list'

In [15]:
print(q.contents[0])

<rebound.Orbit instance, a=2.1249758803e-314 e=2.1254975504e-314 inc=2.125347825e-314 Omega=2.1250259193e-314 omega=2.12462692e-314 f=0.0>


In [16]:
print(q[0])

In [9]:
print(q.contents[0])

<rebound.Orbit instance, a=2.162923098e-314 e=2.1629231456e-314 inc=2.162923098e-314 Omega=1e-323 omega=0.0 f=0.0>


In [10]:
type(rebound.Orbit)

_ctypes.PyCStructType

In [10]:
from ctypes import *

In [11]:
a = rebound.Orbit*2

In [12]:
type(a)

_ctypes.PyCArrayType

In [10]:
q.dtype

dtype('V256')

In [12]:
import numpy as np
x = np.array([1.,2.,3.], dtype=object)

In [13]:
type(x[0])

float

In [9]:
import numpy as np
a = np.array([1.,2.,3.])

In [10]:
print(a.shape)

(3,)


In [9]:
import numpy as np
a = np.array(test, dtype=object)

In [10]:
type(a[0])

rebound.simulation.Orbit

In [13]:
a = np.array([1.,2.,3.])

In [14]:
type(a[0])

numpy.float64

In [15]:
type(a)

numpy.ndarray

In [16]:
type(a[0]).__name__

'float64'

In [17]:
type(a) == np.ndarray

True

In [18]:
c = np.int64()

In [19]:
type(c)

numpy.int64

In [20]:
type(c).__name__

'int64'

In [21]:
type(a).__name__

'ndarray'